In [1]:
import os
import sys
path=!pwd
sys.path.append(os.path.join(path[0], '..'))
import numpy as np
import math
import pandas as pd
import re
from datetime import date, datetime, timedelta
import json
from abc import ABCMeta, abstractmethod
import boto3
import snowflake.connector

class Credentials(metaclass=ABCMeta):
    pass
    
    
class SSMPSCredentials(Credentials):
    def __init__(self, secretid: str):
        self._secretid = secretid
        self._secrets = {}
        
    def get_keys(self):
        """
        credential fetching 
        """
        _aws_sm_args = {'service_name': 'secretsmanager', 'region_name': 'us-east-1'}
        secrets_client = boto3.client(**_aws_sm_args)
        get_secret_value_response = secrets_client.get_secret_value(SecretId=self._secretid)
        return get_secret_value_response
    
    
class BaseConnector(metaclass=ABCMeta):
    @abstractmethod
    def connect(self):
        raise NotImplementedError

        
class SnowflakeConnector(BaseConnector):
    def __init__(self, credentials: Credentials):
        keys = credentials.get_keys()
        self._secrets = json.loads(keys.get('SecretString', "{}"))

    def connect(self, dbname: str, schema: str = 'DEFAULT'):
        ctx = snowflake.connector.connect(
            user=self._secrets['login_name'],
            password=self._secrets['login_password'],
            account=self._secrets['account'],
            warehouse=self._secrets['warehouse'],
            database=dbname,
            schema=schema
        )
        return ctx

    
def run_query(querystr, ctx):
    cursor_list = ctx.execute_string(
        querystr
        )
    df = pd.DataFrame.from_records(cursor_list[-1].fetchall(), columns=[x[0] for x in cursor_list[-1].description])
    df.columns= df.columns.str.lower()
    return df


## Credentials
SF_CREDS = 'datascience-max-dev-sagemaker-notebooks'

## Snowflake connection 
conn=SnowflakeConnector(SSMPSCredentials(SF_CREDS))
ctx=conn.connect("MAX_PROD","DATASCIENCE_STAGE")
cur = ctx.cursor()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


NoCredentialsError: Unable to locate credentials

In [2]:
SSMPSCredentials(SF_CREDS)

In [ ]:
    select 
        ds.hbo_uuid
        , (case if ds.subscription_cancel_ts>='2022-07-01' then 0 else 1) as is_cancel --canceled in June 
        , ac.pillar_genre as category
        , sum(hb.stream_elapsed_play_seconds/3600) as hours_viewed
    from max_prod.bi_analytics.dimension_subscription ds
    left join max_prod.viewership.max_user_stream_heartbeat hb
        on hb.hbo_uuid = ds.hbo_uuid
    left join max_prod.catalog.airtable_content_strategy ac
        on hb.viewable_id = ac.viewable_id
    where ds.subscription_start_ts <='2022-05-01'
    and ds.subscription_cancel_ts >='2022-06-01'
    and ds.original_territory='HBO MAX DOMESTIC'
    and ds.is_retail = 'TRUE'
    and hb.request_date >='2022-05-01'
    and hb.request_date <='2022-06-01'
    and hb.viewable_id IS NOT NULL 
    and hb.stream_elapsed_play_seconds >= 120
    and hb.video_type = 'main'
    and hb.channel = 'HBO MAX SUBSCRIPTION'
    and hb.country_iso_code in ('US')
    group by 1,2,3
    )
    select 
      uh.category
      , round(uh.hours_viewed, 0) as hours_viewed
      , sum(uh.is_cancel) as total_cancel
      , count(distinct(uh.hbo_uuid)) as total_viewer
    from user_hourview uh
    group by 1,2

In [ ]:
case when fcr.free_paid_ind = 'trial' then 1 else 0 end

In [9]:
%%time
churn_hv_query = '''
with user_hourview as (
    select 
        ds.hbo_uuid
        , case when ds.subscription_cancel_ts>='2022-07-01' then 0 else 1 end as is_cancel --canceled in June 
        , ac.pillar_genre as category
        , sum(hb.stream_elapsed_play_seconds/3600) as hours_viewed
    from max_prod.bi_analytics.dimension_subscription ds
    left join max_prod.viewership.max_user_stream_heartbeat hb
        on hb.hbo_uuid = ds.hbo_uuid
    left join max_prod.catalog.airtable_content_strategy ac
        on hb.viewable_id = ac.viewable_id
    where ds.subscription_start_ts <='2022-05-01'
    and ds.subscription_cancel_ts >='2022-06-01'
    and ds.original_territory='HBO MAX DOMESTIC'
    and ds.is_retail = 'TRUE'
    and hb.request_date >='2022-05-01'
    and hb.request_date <='2022-06-01'
    and hb.viewable_id IS NOT NULL 
    and hb.stream_elapsed_play_seconds >= 120
    and hb.video_type = 'main'
    and hb.channel = 'HBO MAX SUBSCRIPTION'
    and hb.country_iso_code in ('US')
    group by 1,2,3
    )
select 
  uh.category
  , round(uh.hours_viewed, 0) as hours_viewed
  , sum(uh.is_cancel) as total_cancel
  , count(distinct(uh.hbo_uuid)) as total_viewer
from user_hourview uh
group by 1,2
'''

df_hv =  run_query(churn_hv_query, ctx)
print('done')

done
CPU times: user 263 ms, sys: 12.8 ms, total: 275 ms
Wall time: 3min 3s


In [19]:
display(df_hv.sort_values(by=['hours_viewed', 'category']))
display(df_hv.describe())
df_hv[df_hv.hours_viewed==100]

,category,hours_viewed,total_cancel,total_viewer
1340,Action/Adventure,0,23644,413189
10697,Adult Animation,0,23320,403533
11544,Classics,0,10167,212546
1296,Comedy,0,19167,310972
3368,Documentary,0,12666,257679
...,...,...,...,...
12763,Adult Animation,1846690,0,1
4078,Action/Adventure,1973639,0,1
302,Kids/Family,2803616,0,1
4864,Drama,5919098,0,1


,hours_viewed,total_cancel,total_viewer
count,1.317500e+04,13175.000000,1.317500e+04
mean,2.907833e+03,158.117799,3.248825e+03
std,8.521722e+04,1539.894249,3.052761e+04
min,0.000000e+00,0.000000,1.000000e+00
25%,3.090000e+02,0.000000,1.000000e+00
50%,9.080000e+02,0.000000,6.000000e+00
75%,1.694000e+03,3.000000,5.500000e+01
max,6.695214e+06,47486.000000,1.120818e+06


,category,hours_viewed,total_cancel,total_viewer
2376,None,100,1,7
2565,Adult Animation,100,136,2742
2916,Suspense/Thriller,100,0,1
3774,Unscripted,100,32,855
4697,Action/Adventure,100,36,544
5168,Horror,100,2,18
6406,Classics,100,0,15
8636,Drama,100,354,8490
8993,Comedy,100,283,6931
9002,Kids/Family,100,110,1963


In [ ]:


# churn_query = '''
# select 
#     ds.hbo_uuid
#     , se.subscription_id
#     , se.subscription_event_timestamp as churn_date
#     , se.subscription_start_date
#     , se.subscription_month_number
# from max_prod.bi_analytics.subscription_events se
# left join max_prod.bi_analytics.dimension_subscription ds
#     on se.subscription_id = ds.subscription_id 
# where se.subscription_event_timestamp >= '2022-05-01'
# and se.subscription_event_timestamp <= '2022-07-01'
# and se.is_cancel='TRUE' 
# and se.is_paid='TRUE' 
# and se.region='NORTH AMERICA'
# and se.territory = 'HBO MAX DOMESTIC'
# and se.product_code = 'hboMax'
# '''


# hv_query = '''
# select 
#   hb.hbo_uuid
#   , hb.request_date
#   , ac.pillar_genre as category
#   , sum(hb.stream_elapsed_play_seconds)/3600. as hours_viewed
# from max_prod.viewership.max_user_stream_heartbeat hb
# left join max_prod.catalog.airtable_content_strategy ac
#   on hb.viewable_id = ac.viewable_id
# where 
#   hb.request_date >='2022-05-01' 
#   and hb.request_date <='2022-05-02'
#   and hb.viewable_id IS NOT NULL 
#   and hb.stream_elapsed_play_seconds >= 120
#   and hb.video_type = 'main'
#   and hb.channel = 'HBO MAX SUBSCRIPTION'
#   and hb.country_iso_code in ('US')
# group by 1,2,3
# '''



In [4]:
%%time
print(1)

1
CPU times: user 88 µs, sys: 0 ns, total: 88 µs
Wall time: 69.1 µs


In [3]:

# churn_query = '''
# create or replace table max_dev.workspace.title_churn (
#       coalesced_id varchar (255) not null
#     , coalesced_title varchar (255) not null
#     , has_churn integer
#     , hbo_uuid varchar(255) not null
#     , hours_watched double
# ) as (
# with base as (
#       select
#              fcr.product_code
#            , fcr.provider
#            , fcr.subscription_start_date
#            , case
#                  when (fcr.provider in ('google', 'samsung') and
#                        datediff(day, fcr.subscription_expire_date, current_timestamp) <= 2) then subscription_cancel_date
#                  else fcr.subscription_expire_date
#                       end as subscription_expire_date
#            , fcr.subscription_cancel_date
#            , fcr.next_start
#            , case when fcr.free_paid_ind = 'trial' then 1 else 0 end
#                as is_trial
#            , fcr.is_cancel
#            , fcr.provider_user_id
#       from max_prod.bi_analytics.fact_common_receipt fcr
#   ),
#      crs_sub_balance as (
#          select
#                 uuid as hbo_uuid
#               , provider
#               , case when provider in ('dtc', 'google', 'samsung', 'apple')
#                         and to_date(subscription_start_date) >= to_date('2020-05-27') then 'hboMax'
#                     when provider = 'amazon' and to_date(subscription_start_date) >= to_date('2020-11-17') then 'hboMax'
#                     when provider = 'roku' and to_date(subscription_start_date) >= to_date('2020-12-17') then 'hboMax'
#                     when to_date(subscription_start_date) < to_date('2020-05-27') then 'hboNow'
#                         else null end as platform_name
#               , is_trial
#               , is_cancel
#               , provider_user_id
#               , subscription_start_date
#               , subscription_cancel_date
#               , row_number() over (partition by hbo_uuid order by subscription_start_date asc) as tenure_rank
#               , case
#                     when not is_cancel and next_start is not null
#                         then next_start
#                     else subscription_expire_date
#              end     as subscription_expire_date
#         from base as a
#         left join enterprise_data.identity.idgraph_vertex as b
#                     on a.provider_user_id = b.name
#          where 1 = 1
#            and product_code in ('hboMax', 'hboNow')
#            and provider in ('dtc', 'google', 'samsung', 'apple','amazon','roku')
#            and uuid is not null
#            and provider_user_id is not null
#            and platform_name is not null
#      ),
#      previous_expire_table as (
#          select
#                 *
#               , lag(subscription_expire_date, 1)
#                     over (partition by hbo_uuid, provider, platform_name, is_trial order by subscription_start_date asc)
#                         as preivous_expire_date
#               -- if the gap is less than a day, then regard it as a continous session
#               , case
#               -- only takes the autorenewal into account for now
#                     when subscription_start_date = preivous_expire_date then 0
#                     else 1
#                         end as cont_sub_session_ind
#          from crs_sub_balance
#      ),
# -- create sub session ind for combination
#      sub_session_table as (
#          select
#                 *
#               , sum(cont_sub_session_ind)
#                     over (partition by hbo_uuid, provider, platform_name, is_trial order by subscription_start_date asc)
#                         as sub_session_ind
#          from previous_expire_table
#      ),

# -- do combination based on the sub session
#      session_connection_table as (
#          select
#                 hbo_uuid
#               , provider
#               , is_trial
#               , is_cancel
#               , platform_name
#               , sub_session_ind
#               , current_timestamp() as last_update_timestamp
#               , min(subscription_start_date)  as subscription_start_timestamp
#               , max(subscription_expire_date) as subscription_expire_timestamp
#               , max(subscription_cancel_date) as subscription_cancel_timestamp
#          from sub_session_table
#          where tenure_rank != 1
#          group by 1, 2, 3, 4, 5, 6
#      ),
#   churn as (
#     select *, case when subscription_cancel_timestamp between '2021-01-03' and '2021-05-03' then 1 else 0 end as has_churn
#     from session_connection_table as a
#     where 1= 1
#         and (
#           (a.subscription_expire_timestamp < '2021-01-03') or (a.subscription_start_timestamp >= '2021-05-03')
#         ) = FALSE
#     )
# select 
#       coalesce(a.series_id, a.viewable_id) as coalesced_id
#     , coalesce(a.series_title_short, a.asset_title_short) as coalesced_title
#     , c.has_churn
#     , c.hbo_uuid
#     , sum(hb.stream_elapsed_play_seconds)/3600. as hours_watched
# from churn c
# join max_prod.viewership.max_user_stream_heartbeat hb
#     on c.hbo_uuid = hb.hbo_uuid and hb.request_time_gmt between '2021-01-03' and '2021-05-03'
# join max_prod.catalog.asset_dim a
#     on hb.viewable_id = a.viewable_id
# where 1 = 1
#     and hb.stream_elapsed_play_seconds > 120
#     and a.asset_type in ('FEATURE', 'ELEMENT')
# group by 1, 2, 3, 4
# )
# '''


# df_churn = run_query(churn_query, ctx)

KeyboardInterrupt: 

In [6]:
churn_query = '''
select * from max_prod.viewership.max_user_stream_heartbeat limit 5
'''

df_churn = run_query(churn_query, ctx)

In [7]:
# create or replace table max_dev.workspace.zzz_je_title_churn (
#       coalesced_id varchar (255) not null
#     , coalesced_title varchar (255) not null
#     , has_churn integer
#     , hbo_uuid varchar(255) not null
#     , hours_watched double
# ) as (
# with base as (
#       select
#              product_code
#            , provider
#            , subscription_start_date
#            , case
#                  when (provider in ('google', 'samsung') and
#                        datediff(day, subscription_expire_date, current_timestamp) <= 2) then subscription_cancel_date
#                  else subscription_expire_date
#                       end as subscription_expire_date
#            , subscription_cancel_date
#            , next_start
#            , case when free_paid_ind = 'trial' then 1 else 0 end
#                as is_trial
#            , is_cancel
#            , provider_user_id

#       from max_prod.bi_analytics.fact_common_receipt
#   ),
#        crs_sub_balance as (
#            select
#                   uuid as hbo_uuid
#                 , provider
#                 , case when provider in ('dtc', 'google', 'samsung', 'apple')
#                           and to_date(subscription_start_date) >= to_date('2020-05-27') then 'hboMax'
#                       when provider = 'amazon' and to_date(subscription_start_date) >= to_date('2020-11-17') then 'hboMax'
#                       when provider = 'roku' and to_date(subscription_start_date) >= to_date('2020-12-17') then 'hboMax'
#                       when to_date(subscription_start_date) < to_date('2020-05-27') then 'hboNow'
#                           else null end as platform_name
#                 , is_trial
#                 , is_cancel
#                 , provider_user_id
#                 , subscription_start_date
#                 , subscription_cancel_date
#                 , row_number() over (partition by hbo_uuid order by subscription_start_date asc) as tenure_rank
#                 , case
#                       when not is_cancel and next_start is not null
#                           then next_start
#                       else subscription_expire_date
#                end     as subscription_expire_date
#           from base as a
#                   left join enterprise_data.identity.idgraph_vertex as b
#                       on a.provider_user_id = b.name
#            where 1 = 1
#              and product_code in ('hboMax', 'hboNow')
#              and provider in ('dtc', 'google', 'samsung', 'apple','amazon','roku')
#              and uuid is not null
#              and provider_user_id is not null
#              and platform_name is not null
#        ),
#        previous_expire_table as (
#            select
#                   *
#                 , lag(subscription_expire_date, 1)
#                       over (partition by hbo_uuid, provider, platform_name, is_trial order by subscription_start_date asc)
#                           as preivous_expire_date
#                 -- if the gap is less than a day, then regard it as a continous session
#                 , case
#                 -- only takes the autorenewal into account for now
#                       when subscription_start_date = preivous_expire_date then 0
#                       else 1
#                           end as cont_sub_session_ind
#            from crs_sub_balance
#        ),
# -- create sub session ind for combination
#        sub_session_table as (
#            select
#                   *
#                 , sum(cont_sub_session_ind)
#                       over (partition by hbo_uuid, provider, platform_name, is_trial order by subscription_start_date asc)
#                           as sub_session_ind
#            from previous_expire_table
#        ),

# -- do combination based on the sub session
#        session_connection_table as (
#            select
#                   hbo_uuid
#                 , provider
#                 , is_trial
#                 , is_cancel
#                 , platform_name
#                 , sub_session_ind
#                 , current_timestamp() as last_update_timestamp
#                 , min(subscription_start_date)  as subscription_start_timestamp
#                 , max(subscription_expire_date) as subscription_expire_timestamp
#                 , max(subscription_cancel_date) as subscription_cancel_timestamp
#            from sub_session_table
#            where tenure_rank != 1
#            group by 1, 2, 3, 4, 5, 6
#        ),
# churn as (
#   select *, case when subscription_cancel_timestamp between '2021-01-03' and '2021-05-03' then 1 else 0 end as has_churn
#   from session_connection_table as a
#   where 1= 1
#       and (
#         (a.subscription_expire_timestamp < '2021-01-03') or (a.subscription_start_timestamp >= '2021-05-03')
#       ) = FALSE
# )
# select 
#       coalesce(a.series_id, a.viewable_id) as coalesced_id
#     , coalesce(a.series_title_short, a.asset_title_short) as coalesced_title
#     , c.has_churn
#     , c.hbo_uuid
#     , sum(hb.stream_elapsed_play_seconds)/3600. as hours_watched
# from churn c
# join max_prod.viewership.max_user_stream_heartbeat hb
#     on c.hbo_uuid = hb.hbo_uuid and hb.request_time_gmt between '2021-01-03' and '2021-05-03'
# join max_prod.catalog.asset_dim a
#     on hb.viewable_id = a.viewable_id
# where 1 = 1
#     and hb.stream_elapsed_play_seconds > 120
#     and a.asset_type in ('FEATURE', 'ELEMENT')
# group by 1, 2, 3, 4
# )

# with unique_base as (
#     select
#         coalesce(m.viewable_series_id, m.viewable_id) as coalesced_id
#         , m.pillar_genre
#     from MAX_PROD.CATALOG.AIRTABLE_CONTENT_STRATEGY m
#     group by 1, 2
# )
# select 
#       m.pillar_genre 
#     , avg(c.hours_watched) as avg_hours
#     , count(distinct c.hbo_uuid) as number_users
#     , count(distinct c.coalesced_id) as total_titles
# //    , round(c.hours_watched, 0) as hours_watched
# //    , sum(c.has_churn) / (1.0*count(c.has_churn)) as churn_rate
# //    , count(distinct c.hbo_uuid) as num_subs
# from unique_base m
# join max_dev.workspace.zzz_je_title_churn c
#     on m.coalesced_id = c.coalesced_id
# where 1 = 1
#     and c.hours_watched <= 100
# group by 1
# order by pillar_genre --, hours_watched asc

,hbo_uuid,user_id,hurley_profile_id,media_framework_guid,viewable_id,stream_min_timestamp_gmt,stream_max_timestamp_gmt,server_stream_start_time,server_stream_end_time,stream_elapsed_play_seconds,...,local_request_date,country_iso_code,channel,stream_type,consent_flags,service_type,home_country,session_country,local_country_date,local_country_time
0,5570ff2243eb4cc2aead7078ead483fac487739f795f10...,gJxPOlN/j1XuQHbsrLeNdDF496KSSh/pL93RX1nNMnE=,W3J7+uPbNhp5EOU6Ibgh5bI2kaRY0yLoi9UANIIJNhc=,OGRVUzlBenBaUXA5M2FSZk5rTE9qMmY2bGQwcHBoV0tmcT...,GVU4QggRzzY7DwvwIAb9a,2021-11-01 17:17:42.713,2021-11-01 18:10:19.830,2021-11-01 17:17:44.043,2021-11-01 18:10:20.188,2809.000,...,2021-11-01,US,HBO MAX SUBSCRIPTION,SVOD,"{\n ""consentFlags"": {\n ""allowFunctionalCo...",SVOD,US,US,2021-11-01,2021-11-01 10:17:44.043
1,ef4caae8d455ca13c27fc540f4d043c9e116c11837b16f...,UUBjnmNMKgHqd+XjkFG0+IFH8RfDEBsnaa8n1P/0HnE=,w7LTE8rGkHbyjT1Iq5UZSj9PAsT1+H6gaGiMDuWUyPw=,ZVFHS0Vua05TVlRsbDlMRUxyNlM5bFBwbXNWb3l4dko5dD...,GYAcbFw1MtsJSiAEAAACh,2021-11-01 10:49:06.814,2021-11-01 11:11:10.593,2021-11-01 10:49:07.967,2021-11-01 11:11:11.740,1322.000,...,2021-11-01,US,HBO MAX SUBSCRIPTION,SVOD,"{\n ""consentFlags"": {\n ""allowFunctionalCo...",SVOD,US,US,2021-11-01,2021-11-01 03:49:07.967
2,30e7fd095d1f364fd14a437b18e144d2be31bc71efe82b...,Qfv1j130eqo0aa3X+8hPhgXQW1vRuTT3h23WANOXoLE=,32ExQeaHQeFplisMW/+Xi+H8AExRXcXCqcZgEKYhXjY=,L093akc0QnBxOXMwWC8xb1BscXNoNnc2M0JVREd1VFc5cm...,GYW78XgVxFcN8sAEAAAH8,2021-11-01 05:33:54.119,2021-11-01 05:46:54.993,2021-11-01 05:33:54.155,2021-11-01 05:46:54.976,710.000,...,2021-10-31,MX,HBO MAX SUBSCRIPTION,SVOD,"{\n ""consentFlags"": {\n ""allowFunctionalCo...",SVOD,MX,MX,2021-10-31,2021-10-31 23:33:54.155
3,fbf0d8fe6d15ba09627e3a0edb1a987720420e26c06874...,l9iFLORW/SF9HuIQK4uUW20SXH3k696z9nAlz2CmNoY=,TGKJD0N6V7OU6eMoCpeY1CoAhi07rcnP3qA4cXyEKqk=,MnRmKzRMbXZBd0V2RThwcnFMMmpZL0d0d2tiaCtkMUk3bV...,GX-nzMwc-EaCFpgEAAAMh,2021-11-01 01:06:22.786,2021-11-01 01:13:50.827,2021-11-01 01:06:23.043,2021-11-01 01:13:51.035,437.000,...,2021-10-31,US,HBO MAX SUBSCRIPTION,SVOD,"{\n ""consentFlags"": {\n ""allowFunctionalCo...",SVOD,US,US,2021-10-31,2021-10-31 18:06:23.043
4,a4803a12fa16cc7821a0061c6fc6263307a5a2e83f5421...,naqCNlShnalhtoAMMcX1HXWjPGgXm1BI486OMgBgxDc=,JcwWTKx9U/8ar4F9ijbZ/YC3TjbzBMtrAogDKlWLn4k=,QlFiSm5WazhXSkJLa0Q0WVRBT3ZsdDJSM25ybWxVNFFkVk...,GXdbTqwqvoMPCwwEAACgN,2021-11-01 04:24:15.040,2021-11-01 04:47:26.964,2021-11-01 04:24:15.146,2021-11-01 04:47:27.047,1390.000,...,2021-10-31,US,HBO MAX SUBSCRIPTION,SVOD,"{\n ""consentFlags"": {\n ""allowFunctionalCo...",SVOD,US,US,2021-10-31,2021-10-31 21:24:15.146
